# Объединение DataFrame: знакомимся с новыми данными

##### С КАКИМИ ДАННЫМИ МЫ РАБОТАЕМ

В этой части модуля мы будем работать с популярным датасетом MovieLens, в котором собраны логи некоторой рекомендательной системы фильмов.

Итак, представим, что нам надо получить единую таблицу, в которой будут собраны рейтинги, даты выставления рейтингов, а также информация о фильмах. Вот как мы будем действовать:

1. Склеим таблицы ratings1 и ratings2 в единую структуру.

2. К полученной таблице с рейтингами присоединим столбец с датой выставления рейтинга.

3. Присоединим к нашей таблице информацию о названиях и жанрах фильмов.

##### Задачи

Значения из какого столбца таблиц ratings1 и ratings2 можно расшифровать с помощью таблицы movies?

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display

In [9]:
ratings1 = pd.read_csv('data/movies/ratings1.csv')
display(ratings1.head())
ratings2 = pd.read_csv('data/movies/ratings2.csv')
display(ratings2.head())
movies = pd.read_csv('data/movies/movies.csv')
display(movies.head())
dates = pd.read_csv('data/movies/dates.csv')
display(dates.head())

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


,userId,movieId,rating
0,274,5621,2.0
1,274,5630,3.0
2,274,5667,3.5
3,274,5679,3.5
4,274,5690,3.0


,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,date
0,2000-07-30 18:45:03
1,2000-07-30 18:20:47
2,2000-07-30 18:37:04
3,2000-07-30 19:03:35
4,2000-07-30 18:48:51


Сколько уникальных фильмов представлено в таблице movies

In [10]:
movies.nunique()

movieId    9742
title      9737
genres      951
dtype: int64

Сколько уникальных пользователей в таблице ratings1

In [11]:
ratings1.nunique()

userId      274
movieId    6219
rating       10
dtype: int64

В каком году было выставлено больше всего оценок

In [22]:
pd.to_datetime(dates['date']).dt.year.value_counts()

2000    10061
2017     8198
2007     7114
2016     6703
2015     6616
2018     6418
1996     6040
2005     5813
2012     4656
2008     4351
2009     4158
2006     4059
2003     4014
2001     3922
2002     3478
2004     3279
1999     2439
2010     2301
1997     1916
2011     1690
2013     1664
2014     1439
1998      507
Name: date, dtype: int64

# Объединение DataFrame: concat

Следуя нашему плану объединения таблиц, первым делом мы должны склеить таблицы ratings1 и ratings2 по строкам.

Для этого воспользуемся встроенной функцией Pandas concat(), которая позволяет склеивать (конкатенировать) таблицы как по строкам, так и по столбцам.

Основные параметры функции 

**concat()**

* objs — список объектов DataFrame ([df1, df2,…]), которые должны быть сконкатенированы;
* axis — ось определяет направление конкатенации: 0 — конкатенация по строкам (по умолчанию), 1 — конкатенация по столбцам;
* join — либо inner (пересечение), либо outer (объединение); рассмотрим этот момент немного позже;
* ignore_index — по умолчанию установлено значение False, которое позволяет значениям индекса оставаться такими, какими они были в исходных данных. Если установлено значение True, параметр будет игнорировать исходные значения и повторно назначать значения индекса в последовательном порядке.

Для корректной конкатенации по строкам объединяемые таблицы должны иметь одинаковую структуру — идентичное число и имена столбцов

**concat** является функцией библиотеки, а не методом DataFrame. Поэтому её вызов осуществляется как **pd.concat(...)**

In [7]:
# Склеим таблицы ratings1 и ratings2
ratings = pd.concat([ratings1, ratings2]) # Для этого передадим их в списке в функцию concat()
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
60831,610,166534,4.0
60832,610,168248,5.0
60833,610,168250,5.0
60834,610,168252,5.0


**На первый взгляд может показаться, что всё прошло успешно, однако если мы посмотрим на индексы таблицы, то заметим, что их нумерация не совпадает с количеством строк в таблице, то есть у нас есть дублирующиеся индексы. Это может привести к некорректному объединению таблиц по ключевым столбцам**

Это связано с тем, что по умолчанию concat сохраняет первоначальные индексы объединяемых таблиц, а обе наши таблицы индексировались, начиная от 0. Чтобы создать новые индексы, нужно выставить параметр ignore_index на True

In [8]:
ratings = pd.concat(
    [ratings1, ratings2],
    ignore_index=True
)
display(ratings)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100832,610,166534,4.0
100833,610,168248,5.0
100834,610,168250,5.0
100835,610,168252,5.0


Казалось бы, совсем другое дело! Но это ещё не всё. Давайте узнаем количество строк в таблицах ratings и dates, ведь нам предстоит объединять их по столбцам

In [10]:
print('Число строк в таблице ratings: ', ratings.shape[0])
print('Число строк в таблице dates: ', dates.shape[0])
print(ratings.shape[0] == dates.shape[0])

Число строк в таблице ratings:  100837
Число строк в таблице dates:  100836
False


In [11]:
# Выведем последнюю строку таблицы ratings1 и первую строку таблицы ratings2
display(ratings1.tail(1))
display(ratings2.head(1))

,userId,movieId,rating
40000,274,5621,2.0


,userId,movieId,rating
0,274,5621,2.0


**drop_duplicates()**

Чтобы очистить таблицу от дублей, мы можем воспользоваться методом DataFrame drop_duplicates(), который удаляет повторяющиеся строки в таблице. Не забываем обновить индексы после удаления дублей, выставив параметр ignore_index в методе drop_duplicates() на значение True:

In [12]:
ratings = ratings.drop_duplicates(ignore_index=True)
print('Число строк в таблице ratings: ', ratings.shape[0])

Число строк в таблице ratings:  100836


In [13]:
# Конкатенируем таблицы ratings и dates по столбцам
ratings_dates = pd.concat([ratings, dates], axis=1)
display(ratings_dates.tail(7))

,userId,movieId,rating,date
100829,610,164179,5.0,2017-05-03 21:07:11
100830,610,166528,4.0,2017-05-04 06:29:25
100831,610,166534,4.0,2017-05-03 21:53:22
100832,610,168248,5.0,2017-05-03 22:21:31
100833,610,168250,5.0,2017-05-08 19:50:47
100834,610,168252,5.0,2017-05-03 21:19:12
100835,610,170875,3.0,2017-05-03 21:20:15


Вам необходимо написать функцию concat_user_files(path), параметром которой является path - путь до директории. 
Функция должна объединить информацию из предоставленных вам файлов в один DataFrame и вернуть его. Не забудьте обновить индексы результирующей таблицы после объединения. Учтите тот момент, что в результате объединения могут возникнуть дубликаты, от которых необходимо будет избавиться. 

In [14]:
# код не будет работать из-за отсутствия файлов в системе
import pandas as pd
import os # https://pythonworld.ru/moduli/modul-os.html - подробнее про этот модуль

def concat_users_files(path):
    files_list = sorted(os.listdir(path)) # сортируем список со всекми файлами в директории паф
    df = None # задаём пустой нулевой датафрейм
    for file in files_list: # работаем столько итераций, сколько файлов директории
        df_new = pd.read_csv(f'{path}{file}') # считываем полный путь файла с его названием и создаём временный датафрейм
        df = pd.concat([df,df_new],ignore_index=True) # конкатенируем нулевой и временный датафреймы внутрь нулевого
    df = df.drop_duplicates(ignore_index=True) # избавляемся от возможных дубликатов
    return df
if __name__ == '__main__':
    data = concat_users_files('./Root/users/')

FileNotFoundError: [WinError 3] Системе не удается найти указанный путь: './Root/users/'

# Объединение DataFrame: join, merge

У таблиц ratings и movies есть общий столбец movieId, который каждому фильму из таблицы movies ставит в соответствие поставленные ему оценки из таблицы ratings. Мы хотим объединить их в единую структуру согласно этому соответствию. Объединения такого рода часто называют **объединением по ключевому столбцу**

##### ТИПЫ ОБЪЕДИНЕНИЙ

Типы объединений в Pandas тесно связаны с операцией join из SQL, которую мы будем рассматривать в курсе в дальнейшем.

Они представлены на схеме ниже в виде кругов Эйлера.

![Типы объединений](https://lms.skillfactory.ru/assets/courseware/v1/bb88a7e1138505f1c3b4ce9282a14742/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md12_7_1.png "join")

**inner (внутреннее)**

При использовании такого типа объединения в результирующей таблице остаются только те записи, которые есть в обеих таблицах.  
Аналогия в теории множеств  
Пересечение (intersection) множеств А и В.  
Строки, для которых совпадение не было найдено, удаляются.

**outer (внешнее)**

Данный тип делится на три подтипа:  
**full** — используется как outer по умолчанию, объединяет все варианты в обеих таблицах.  
Аналогия в теории множеств  
**left** — для всех записей из «левой» таблицы (например, ratings) ведётся поиск соответствий в «правой» (например, movies). В результирующей таблице останутся только те значения, которым были найдены соответствия, то есть только значения из ratings.  
Аналогия в теории множеств  
Вычитание (difference) множества B из результата объединения (union) множеств А и В.  
**right** — аналогично предыдущему, но остаются значения только из «правой» таблицы. 
Аналогия в теории множеств  
Вычитание (difference) множества А из результата объединения (union) множеств А и В.

##### МЕТОД ОБЪЕДИНЕНИЯ JOIN

Для объединения двух таблиц по индексам используется метод DataFrame

**join()**

Однако данный метод можно применить и для того, чтобы объединить таблицы по ключевому столбцу (в нашем случае это movieId).

* other — таблица, которую мы присоединяем. При объединении она является «правой», а исходная таблица, от имени которой вызывается метод, является «левой».
* how — параметр типа объединения. Он может принимать значения 'inner', 'left' (left outer), 'right' (right outer), и 'outer' (full outer). По умолчанию параметр установлен на 'left'.
* on — параметр, который определяет, по какому столбцу в «левой» таблице происходит объединение по индексам из «правой».
* lsuffix и rsuffix — дополнения (суффиксы) к названиям одноимённых столбцов в «левой» и «правой» таблицах.

Ниже представлена общая схема работы метода join() в зависимости от типа объединения:

![Работа JOIN](https://lms.skillfactory.ru/assets/courseware/v1/e16f8a1193113a12294003598768bbc0/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md12_7_2.png "join")


In [19]:
joined_false = ratings_dates.join(
    movies,
    rsuffix='_right',
    how='right'
)
display(joined_false)

,userId,movieId,rating,date,movieId_right,title,genres
0,1,1,4.0,2000-07-30 18:45:03,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,2,Jumanji (1995),Adventure|Children|Fantasy
2,1,6,4.0,2000-07-30 18:37:04,3,Grumpier Old Men (1995),Comedy|Romance
3,1,47,5.0,2000-07-30 19:03:35,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,1,50,5.0,2000-07-30 18:48:51,5,Father of the Bride Part II (1995),Comedy
...,...,...,...,...,...,...,...
9737,64,3481,4.0,2006-10-22 12:37:45,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,64,3489,3.0,2006-10-22 23:28:09,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,64,3499,4.5,2006-10-22 23:26:41,193585,Flint (2017),Drama
9740,64,3510,3.0,2006-10-22 23:27:26,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


При объединении таблиц по индексам в результирующую таблицу попали все строки из «левой» таблицы, а недостающие строки из «правой» были заполнены пропусками. Так работает тип объединения left.

Однако это не тот результат, который мы хотели, ведь мы не получили соответствия фильмов и их рейтингов. Чтобы совместить таблицы по ключевому столбцу с помощью метода join(), необходимо использовать ключевой столбец в «правой» таблице в качестве индекса. Это можно сделать с помощью метода 

**set_index()**

Также необходимо указать название ключа в параметре on.

In [20]:
joined = ratings_dates.join(
    movies.set_index('movieId'),
    on='movieId',
    how='left'
)
display(joined.head())

,userId,movieId,rating,date,title,genres
0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,2000-07-30 19:03:35,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,2000-07-30 18:48:51,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


В результате такого объединения мы для каждого идентификатора фильма movieId в таблице ratings_dates нашли совпадающий с ним идентификатор movieId в таблице movies и присоединили информацию о самом фильме (title и genres). Это как раз то, что было нам нужно.

##### МЕТОД ОБЪЕДИНЕНИЯ MERGE